<a href="https://colab.research.google.com/github/satyam-tomar/neural-networks/blob/main/NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
inputs = [1,2,3]
weights = [0.1, 0.2, 0.3]
bias = 2

output = inputs[0]*weights[0] + inputs[1]*weights[1] + inputs[2]*weights[2] + bias
print(output)

3.4


In [7]:
inputs = [1,2,3,2.5]
weights1 = [0.2, 0.8, -0.5, 1.0]
weights2 = [0.5, -0.91, 0.26, -0.5]
weights3 = [-0.26, -0.27, 0.17, 0.87]

bias1 = 2
bias2 = 3
bias3 = 0.5

output = [inputs[0]*weights1[0] + inputs[1]*weights1[1] + inputs[2]*weights1[2] + inputs[2]*weights1[3] + bias1,
          inputs[0]*weights2[0] + inputs[1]*weights2[1] + inputs[2]*weights2[2] + inputs[2]*weights2[3] + bias2,
          inputs[0]*weights3[0] + inputs[1]*weights3[1] + inputs[2]*weights3[2] + inputs[2]*weights3[3] + bias3]

print(output)

[5.3, 0.96, 2.82]


In [6]:
inputs = [1,2,3,2.5]

weights = [[0.2, 0.8, -0.5, 1.0],
           [0.5, -0.91, 0.26, -0.5],
           [-0.26, -0.27, 0.17, 0.87]]

biases = [2,3,0.5]

layer_output = []
for neuron_weights, neuron_bias in zip(weights, biases):
  neural_output = 0
  for n_input, weight in zip(inputs, neuron_weights):
    neural_output += n_input*weight
  neural_output += neuron_bias
  layer_output.append(neural_output)

print(layer_output)



[4.8, 1.21, 2.385]


In [9]:
inputs = [1,2,3,2.5]

weights = [[0.2, 0.8, -0.5, 1.0],
           [0.5, -0.91, 0.26, -0.5],
           [-0.26, -0.27, 0.17, 0.87]]

biases = [2,3,0.5]

some_value = 0.5
weights = -0.7
bias = 0.7

print(some_value*weights)
print(some_value+bias)

-0.35
1.2


In [10]:
import numpy as np

inputs = [1,2,3,2.5]

weights = [0.2, 0.8, -0.5, 1.0]

bias = 2

output = np.dot(weights, inputs) + bias
print(output)



4.8


In [14]:
import numpy as np

inputs = [1,2,3,2.5]

weights = [[0.2, 0.8, -0.5, 1.0],
           [0.5, -0.91, 0.26, -0.5],
           [-0.26, -0.27, 0.17, 0.87]]

biases = [2,3,0.5]

# layer_output = []
# for neuron_weights, neuron_bias in zip(weights, biases):
#   neural_output = np.dot(neuron_weights, inputs) + neuron_bias
#   layer_output.append(neural_output)
# print(layer_output)


output = np.dot(weights, inputs) + biases
print(output)

[4.8   1.21  2.385]


In [1]:
import numpy as np

inputs = [[1,2,3,2.5],
          [2.0, 5.0, -1.0, 2.0],
          [-1.5, 2.7, 3.3, -0.8]]

weights = [[0.2, 0.8, -0.5, 1.0],
           [0.5, -0.91, 0.26, -0.5],
           [-0.26, -0.27, 0.17, 0.87]]

biases = [2,3,0.5]

output = np.dot(inputs , np.array(weights).T) +biases
print(output)
print("this is the output of neural network")

[[ 4.8    1.21   2.385]
 [ 8.9   -1.81   0.2  ]
 [ 1.41   1.051  0.026]]
this is the output of neural network
